## Introduction 
Autoencoder are special type of deep learning architecture that consist of two networks encoder and decoder.
The encoder, through a series of CNN and downsampling, learns a reduced dimensional representation of the input data while decoder  through the use of CNN and upsampling, attempts to regenerate the data from the these representations. A well-trained decoder is able to regenerated data that is identical or as close as possible to the original input data.
Autoencoder are generally used for anamoly detection, denoising image, colorizing the images. Here, i am going to colorize the landscape images using autoencoder.

<img src = 'https://miro.medium.com/max/600/1*nqzWupxC60iAH2dYrFT78Q.png' >

## Image Colorization
Image colorization using different softwares require large amount of human effort, time and skill.But special type of deep learning architecture called autoencoder has made this task quiet easy. Automatic image colorization often involves the use of a class of convolutional neural networks (CNN) called autoencoders. These neural networks are able to distill the salient features of an image, and then regenerate the image based on these learned features. 

<img src = "https://tinyclouds.org/colorize/best/6.jpg">

## Import necessary libraries

In [ ]:
import numpy as np
import tensorflow as tf
import keras
import cv2
from keras.layers import MaxPool2D,Conv2D,UpSampling2D,Input,Dropout
from keras.models import Sequential
from keras.utils import img_to_array, plot_model
from keras.utils.vis_utils import plot_model
import os
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import gc

### Getting landscape image data,resizing them and appending in array
To get the image in sorted order i have defined the function sorted_alphanumeric. Here, I have used open cv library to read and resize images. Finally images are normalized and are converted to array and are appended in empty list

In [ ]:
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)
# defining the size of the image
SIZE = 256
color_img = []
gray_img = []
path = 'E:/TER/LandscapeDataResize/color'
files = os.listdir(path)
files = sorted_alphanumeric(files)
for i in tqdm(files):    
        if i == '7200.jpg':
            break
        else:    
            img = cv2.imread(path + '/'+i,1)
            # open cv reads images in BGR format so we have to convert it to LAB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
            #resizing image
            #分离图像通道
            l, a, b = cv2.split(img)
            xx=b
            #归一化
            l = (l/ 255.0)
            a = (a/ 255.0)
            b = (b/ 255.0)
            #合并通道
            img = cv2.merge([l, a, b])
            color_img.append(img_to_array(img))
            gray_img.append(img_to_array(l))
gc.collect()

### Plotting Color image and it's corresponding grayscale image

In [ ]:
# defining function to plot images pair
def plot_images(color,grayscale):
    plt.figure(figsize = (10,10))
    plt.subplot(1,2,1)
    plt.title('color')
    l, a, b = cv2.split(np.array(color))
    #归一化
    l = (l.astype('float32')* 100)
    a = (a.astype('float32')* 255.0)-128
    b = (b.astype('float32')* 255.0)-128
    #合并通道
    color = cv2.merge([l, a, b])
    plt.imshow(cv2.cvtColor(color, cv2.COLOR_Lab2RGB))
    plt.subplot(1,2,2)
    plt.title('gray')
    plt.imshow(grayscale, cmap = 'gray')
    plt.show()

**Plotting image pair**

In [ ]:
for i in range(3,10):
     plot_images(color_img[i],gray_img[i])

### Slicing and reshaping
Out of 5000 images I have sliced them to two part. train images consist 4000 images  while test images contains 1000 images.
After slicing the image array, I reshaped them so that images can be fed directly into our encoder network

In [ ]:
train_gray_image = gray_img[:4000]
train_color_image = color_img[:4000]

test_gray_image = gray_img[4000:]
test_color_image = color_img[4000:]
del gray_img
del color_img
gc.collect()
# reshaping
train_g = np.reshape(train_gray_image,(len(train_gray_image),SIZE,SIZE,1))
train_c = np.reshape(train_color_image, (len(train_color_image),SIZE,SIZE,3))
print('Train color image shape:',train_c.shape)


test_gray_image = np.reshape(test_gray_image,(len(test_gray_image),SIZE,SIZE,1))
test_color_image = np.reshape(test_color_image, (len(test_color_image),SIZE,SIZE,3))
print('Test color image shape',test_color_image.shape)

## Defining our model
Encoder layer of our model consist blocks of Convolution layer with different number of kernel and kernel_size. Here, Convolution is used for downsampling.
Similary, Decoder layer of our model consist of  transpose convolution layer with different kernel size. Here, Decoder layer upsample image downsampled by encoder.
Since there is feature loss between the encoder and decoder layers so inorder to prevent feature loss i have concatenate corresponding encoder and decoder layers. Check U_Net architecture for better understanding......

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
  
  result = tf.keras.Sequential()
  result.add(
      tf.keras.layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer='he_normal', use_bias=False))

  if apply_batchnorm:
    result.add(tf.keras.layers.BatchNormalization())

  result.add(tf.keras.layers.LeakyReLU())

  return result

def upsample(filters, size, apply_dropout=False):
  

  result = tf.keras.Sequential()
  result.add(
    tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                    padding='same',
                                    kernel_initializer='he_normal',
                                    use_bias=False))

  result.add(tf.keras.layers.BatchNormalization())

  if apply_dropout:
      result.add(tf.keras.layers.Dropout(0.5))

  result.add(tf.keras.layers.ReLU())

  return result  

In [ ]:
from keras import layers
def down(filters , kernel_size, apply_batch_normalization = True):
    downsample = tf.keras.models.Sequential()
    downsample.add(layers.Conv2D(filters,kernel_size,padding = 'same', strides = 2))
    if apply_batch_normalization:
        downsample.add(layers.BatchNormalization())
    downsample.add(keras.layers.LeakyReLU())
    return downsample


def up(filters, kernel_size, dropout = False):
    upsample = tf.keras.models.Sequential()
    upsample.add(layers.Conv2DTranspose(filters, kernel_size,padding = 'same', strides = 2))
    if dropout:
        upsample.dropout(0.2)
    upsample.add(keras.layers.LeakyReLU())
    return upsample

In [ ]:
def autoencoder_lab():
    inputs = tf.keras.layers.Input(shape=[256,256,1])

    down_stack = [
        downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
        downsample(128, 4), # (bs, 64, 64, 128)
        downsample(256, 4), # (bs, 32, 32, 256)
        downsample(512, 4), # (bs, 16, 16, 512)
        downsample(512, 4), # (bs, 8, 8, 512)
        downsample(512, 4), # (bs, 4, 4, 512)
        downsample(512, 4), # (bs, 2, 2, 512)
        downsample(512, 4), # (bs, 1, 1, 512)
    ]

    up_stack = [
        upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
        upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
        upsample(512, 4), # (bs, 16, 16, 1024)
        upsample(256, 4), # (bs, 32, 32, 512)
        upsample(128, 4), # (bs, 64, 64, 256)
        upsample(64, 4), # (bs, 128, 128, 128)
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = tf.keras.layers.Conv2DTranspose(3, 4,
                                        strides=2,
                                        padding='same',
                                        kernel_initializer=initializer,
                                        activation='tanh') # (bs, 256, 256, 3)

    x = inputs

    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = tf.keras.layers.Concatenate()([x, skip])

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)


In [ ]:
autoencoder_lab =autoencoder_lab()
autoencoder_lab.summary()
plot_model(autoencoder_lab, to_file='autoencoder_lab.png', show_shapes=True, show_layer_names=True,rankdir='TB', dpi=320)


### Fitting our model

In [ ]:
autoencoder_lab.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
              metrics = ['acc'])
results=autoencoder_lab.fit(train_g, train_c, epochs = 100, batch_size = 16, verbose = 1)
autoencoder_lab.save('E:/TER/model/autoencoder_lab.h5')

In [ ]:
autoencoder_lab.evaluate(test_gray_image,test_color_image)
plt.plot(results.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.savefig('./loss_autoencoder_lab.png')
plt.show()

# plotting colorized image along with grayscale and color image

In [ ]:
# defining function to plot images pair
def plot_images(color,grayscale,predicted,n):
    M = tf.keras.losses.MeanSquaredError()
    MSE = M(color, predicted)
    plt.figure(figsize=(15,5))
    l, a, b = cv2.split(np.array(color))
    #归一化
    l = (l.astype('float32')* 100)
    a = (a.astype('float32')* 255.0)-128
    b = (b.astype('float32')* 255.0)-128
    #合并通道
    target_image = cv2.merge([l, a, b])  
    l, a, b = cv2.split(np.array(predicted))
    #归一化
    l = (l.astype('float32')* 100)
    a = (a.astype('float32')* 255.0)-128
    b = (b.astype('float32')* 255.0)-128
    #合并通道
    prediction_image = cv2.merge([l, a, b])
    target_image = cv2.cvtColor(target_image, cv2.COLOR_LAB2RGB)
    prediction_image = cv2.cvtColor(prediction_image, cv2.COLOR_LAB2RGB)    
    plt.imsave('E:/TER/Result_autoencoder_LandscapeDataResize_LAB/output/'+str(n)+'.png', prediction_image)
    display_list = [grayscale, target_image, prediction_image]
    title = ['Input Image', 'Ground Truth', 'Predicted Image in LAB']
    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i],cmap='gray')
        plt.axis('off')
    plt.suptitle("MSE: "+str(MSE.numpy()))
    plt.savefig('E:/TER/Result_autoencoder_LandscapeDataResize_LAB/plot/'+str(n)+'.png')
    plt.show()

if not os.path.exists('E:/TER/Result_autoencoder_LandscapeDataResize_LAB/output/'):
    os.makedirs('E:/TER/Result_autoencoder_LandscapeDataResize_LAB/output/')
if not os.path.exists('E:/TER/Result_autoencoder_LandscapeDataResize_LAB/plot/'):
    os.makedirs('E:/TER/Result_autoencoder_LandscapeDataResize_LAB/plot/')

for i in range(319):
    predicted = np.clip(autoencoder_lab.predict(test_gray_image[i].reshape(1,SIZE, SIZE,1)),0.0,1.0).reshape(SIZE, SIZE,3)
    plot_images(test_color_image[i],test_gray_image[i],predicted,i)

# Thanks for your visit.
## Any suggestions to improve this model is highly appreciated.
# Feel free to  comment